<a href="https://colab.research.google.com/github/awaw24/Metaheurystyki/blob/main/Metaheurystyki_Lab_3_Zad_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ponizszy kod Python implementuje wariant II algorytmu „małych światów” z możliwością konfiguracji dowolnych operatorów lokalnych i dalekiego zasięgu oraz zawiera moduł eksperymentów umożliwiający łatwe dodawanie kolejnych funkcji testowych i wariantów operatorów.

In [ ]:
import numpy as np
import pandas as pd

!pip install ace_tools_open

def rastrigin(x):
    n = len(x)
    return 10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

def cauchy_mutation(x, scale=1.0):
    return x + scale * np.random.standard_cauchy(size=x.shape)

def normal_mutation(x, sigma=0.1):
    return x + np.random.normal(scale=sigma, size=x.shape)

def uniform_mutation(x, bound=0.5):
    return x + np.random.uniform(-bound, bound, size=x.shape)

def small_world_algorithm(func, dim, bounds, n=30, max_iter=100, ps=0.7,
                          local_op=None, global_op=None, minimize=True):
    # Initialize population
    pop = np.random.uniform(bounds[0], bounds[1], size=(n, dim))
    best = np.min([func(ind) for ind in pop]) if minimize else np.max([func(ind) for ind in pop])

    for _ in range(max_iter):
        new_pop = pop.copy()
        for i, indy in enumerate(pop):
            if np.random.rand() < ps:
                candidate = local_op(indy)
            else:
                candidate = global_op(indy)
            # Clip to bounds
            candidate = np.clip(candidate, bounds[0], bounds[1])

            if minimize:
                if func(candidate) < func(indy):
                    new_pop[i] = candidate
                    best = min(best, func(candidate))
            else:
                if func(candidate) > func(indy):
                    new_pop[i] = candidate
                    best = max(best, func(candidate))
        pop = new_pop

    return best

# Experiment settings
dim = 2
bounds = (-5.12, 5.12)
runs = 10

operator_sets = {
    "Normal(σ=0.1) local + Normal(σ=1.0) distant": (lambda x: normal_mutation(x, 0.1),
                                                     lambda x: normal_mutation(x, 1.0)),
    "Normal(σ=0.1) local + Cauchy(scale=1.0) distant": (lambda x: normal_mutation(x, 0.1),
                                                        lambda x: cauchy_mutation(x, 1.0)),
    "Uniform(bound=0.5) local + Cauchy(scale=1.0) distant": (lambda x: uniform_mutation(x, 0.5),
                                                             lambda x: cauchy_mutation(x, 1.0))
}

results = []
for name, (loc_op, glob_op) in operator_sets.items():
    bests = []
    for _ in range(runs):
        best_val = small_world_algorithm(rastrigin, dim, bounds, local_op=loc_op, global_op=glob_op)
        bests.append(best_val)
    results.append({
        "Operator Set": name,
        "Avg Best Value": np.mean(bests),
        "Std Best Value": np.std(bests)
    })

df_results = pd.DataFrame(results)

import ace_tools_open as tools; tools.display_dataframe_to_user(name="Rastrigin Function Results (2D)", dataframe=df_results)


Rastrigin Function Results (2D)


Loading ITables v2.4.0 from the internet... (need help?)


Powyższe wyniki w formie tabeli, prezentują średnie najlepsze wartości funkcji Rastrigina (w wymiarze 2) uzyskane przez algorytm „małych światów” (wariant II) po 100 iteracjach, dla każdej z trzech par operatorów mutacji, na podstawie 10 powtórzeń:

* **Normal(σ=0.1) + Normal(1.0)** wypadł "po środku" (≈0.36), za to z najwyższym odchyleniem standardowym.

* **Normal(σ=0.1) + Cauchy(1.0)** osiągnął najlepszy wynik średni (≈0.25), z najniższym odchyleniem standardowym.

* **Uniform(±0.5) + Cauchy(1.0)** dał najgorszy średni rezultat (≈0.59), z nieco niższym odchyleniem standardowym niz druga para.

**Moje wnioski:**

* **Najlepszy średni wynik:** Para Normal(σ = 0.1) lokalny + Cauchy(1.0) dalekiego zasięgu osiągnęła najmniejszą wartość średnią (≈0.25), co oznacza najbliższe przybliżenie do globalnego minimum funkcji.

* **Najmniejsza zmienność:** Ten sam zestaw operatorów cechował się także najniższym odchyleniem standardowym (≈0.32), co świadczy o większej powtarzalności i stabilności wyników między niezależnymi uruchomieniami.

* **Porównanie z Normal+Normal:** Choć para Normal(0.1) + Normal(1.0) miała stosunkowo dobre średnie (≈0.36), jej duże odchylenie (≈0.43) wskazuje na mniejszą przewidywalność wyników.

* **Słabszy wynik przy Uniform+Cauchy:** Wariant Uniform(±0.5) lokalny + Cauchy(1.0) dalekiego zasięgu dał najgorszy średni (≈0.59), a przy tym nadal wysokie odchylenie (≈0.41). Lokalna mutacja jednostajna okazała się tutaj mniej efektywna w refinowaniu rozwiązań.